# 傾向スコアマッチング

傾向スコアマッチングは、観察研究において因果効果を推定するための手法である。
各サンプルが治療を受ける確率（傾向スコア）を推定し、治療群と対照群のサンプル間で、傾向スコアが類似するペアを形成する。これにより、交絡因子（confounders）の影響を制御し、治療効果をより正確に推定することができる。

基本用語
	•	傾向スコア (Propensity Score)
各サンプルが治療を受ける確率  p(T=1 | X)  を表す。ここで、 X  は観測される共変量である。
傾向スコアは通常、ロジスティック回帰などのモデルを用いて推定される。
	•	マッチング
傾向スコアに基づいて、治療群（ T=1 ）の各サンプルに対し、対照群（ T=0 ）から最も類似したサンプルをペアとして選ぶ。これにより、治療群と対照群間の共変量のバランスが改善される。
	•	交絡因子 (Confounders)
治療とアウトカムの両方に影響を与える要因であり、これが存在すると単純な比較では真の因果効果が歪められる可能性がある。傾向スコアマッチングは、これらの交絡因子の影響を調整する目的で用いられる。

どのような時に利用されるか

観察研究では、無作為化比較試験 (RCT) が実施できない場合が多い。そのような場合、治療の割り当てに偏りが生じる可能性があるが、傾向スコアマッチングを用いることで、治療群と対照群の共変量の分布を類似させ、擬似的なランダム化を実現できる。

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.neighbors import NearestNeighbors

# 仮のデータセットを生成する
np.random.seed(42)
n = 1000

# 交絡因子: 年齢、性別、基礎疾患スコア
age = np.random.randint(20, 70, n)
sex = np.random.binomial(1, 0.5, n)
comorbidity = np.random.randint(0, 5, n)

# 傾向スコアを推定するためのロジスティック回帰モデルを仮定（ここではシミュレーション目的）
X = np.column_stack((age, sex, comorbidity))
logit = -3 + 0.05 * age + 0.8 * sex + 0.4 * comorbidity
ps = 1 / (1 + np.exp(-logit))
treatment = np.random.binomial(1, ps)

df = pd.DataFrame({
    'age': age,
    'sex': sex,
    'comorbidity': comorbidity,
    'treatment': treatment,
    'pscore': ps
})

# 治療群と対照群に分割
treated = df[df['treatment'] == 1]
control = df[df['treatment'] == 0]

# 最近傍法による1対1マッチング（傾向スコアのみを使用）
nn = NearestNeighbors(n_neighbors=1)
nn.fit(control[['pscore']])
distances, indices = nn.kneighbors(treated[['pscore']])

# マッチング結果のデータを抽出
control_matched = control.iloc[indices.flatten()].copy()
treated = treated.reset_index(drop=True)
matched = pd.concat([treated, control_matched], axis=0)

# マッチング後の群ごとの平均値を確認
print(matched.groupby('treatment').mean())

                 age       sex  comorbidity    pscore
treatment                                            
0          46.489933  0.617450     2.427852  0.664663
1          48.582215  0.562081     2.288591  0.664936
